In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import os
import shutil
from tensorflow.keras import layers
from tensorflow.keras.layers import Layer
import tensorflow_addons as tfa

# Define constants
num_classes = 2
image_size = 224
input_shape = (image_size, image_size, 3)
labels = ['Malignant', 'Benign']

# Function to load data
def get_data(data_dir):
    images = []
    labels_list = []
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img))[..., ::-1]  # Convert BGR to RGB format
                resized_arr = cv2.resize(img_arr, (image_size, image_size))  # Resize images
                images.append(resized_arr)
                labels_list.append(class_num)
            except Exception as e:
                print(e)
    return np.array(images), np.array(labels_list)

# Load train and test data
train_images, y_train = get_data('.../train')
test_images, y_test = get_data('.../test')

# Normalize data and shuffle
x_train = train_images / 255.0
x_test = test_images / 255.0
from sklearn.utils import shuffle
x_train, y_train = shuffle(x_train, y_train)
x_test, y_test = shuffle(x_test, y_test)

# Define model hyperparameters
learning_rate = 0.0001
weight_decay = 0.00001
dropout = 0.4
batch_size = 16
num_epochs = 15
image_size = 224  
patch_size = 16
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [projection_dim * 2, projection_dim,]
transformer_layers = 4
mlp_head_units = [2048, 1024]
num_classes = 2
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout)(x)  
    return x
#  custom layer "Patches"
class Patches(layers.Layer):
    def __init__(self, patch_size, **kwargs):
        super(Patches, self).__init__(**kwargs)
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

    def get_config(self):
        config = super(Patches, self).get_config()
        config.update({"patch_size": self.patch_size})
        return config

# Define custom layer "PatchEncoder"
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim, **kwargs):
        super(PatchEncoder, self).__init__(**kwargs)
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return layers.Dropout(0.3)(encoded)

    def get_config(self):
        config = super(PatchEncoder, self).get_config()
        config.update({"num_patches": self.num_patches, "projection_dim": self.projection.units})
        return config

def create_cnn_layer(input_shape):
    inputs = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(inputs)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu')(x)  # Add another convolutional layer
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu')(x)  # Add another convolutional layer
    x = tf.keras.layers.Flatten()(x)
    cnn_model = tf.keras.Model(inputs=inputs, outputs=x)
    return cnn_model

# Create the Vision Transformer (ViT) model with CNN
def create_vit_classifier(cnn_model):
    inputs = tf.keras.layers.Input(shape=input_shape)
    cnn_output = cnn_model(inputs)
    patches = Patches(patch_size)(inputs)
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    for _ in range(transformer_layers):
        x1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        x2 = tf.keras.layers.Add()([attention_output, encoded_patches])
        x3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        encoded_patches = tf.keras.layers.Add()([x3, x2])

    representation = tf.keras.layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = tf.keras.layers.Flatten()(representation)
    representation = tf.keras.layers.Dropout(dropout)(representation)
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=dropout)

    combined_features = tf.keras.layers.Concatenate()([cnn_output, features])
    logits = tf.keras.layers.Dense(num_classes)(combined_features)

    model = tf.keras.Model(inputs=inputs, outputs=logits)
    return model


#  the CNN layer
cnn_model = create_cnn_layer(input_shape)

#  the combined model
vit_classifier = create_vit_classifier(cnn_model)


In [ ]:
import shutil

folder_num = 'Folder_name'
data = {
        "learning_rate":learning_rate, 
        "weight_decay":weight_decay, 
        "batch_size":batch_size,
        "num_epochs":num_epochs,
        "image_size":image_size,
        "optimizer":'Adam',
        "patch_size":patch_size,
       "projection_dim":projection_dim,
       "num_heads":num_heads,
       "transformer_units":transformer_units,
    "transformer_layers":transformer_layers,
    "mlp_head_units":mlp_head_units,
    "dropout":dropout,
       }

dir_name = '.../' + str(folder_num)
if not os.path.exists(dir_name):
    os.makedirs(dir_name)
else:
    shutil.rmtree(dir_name)           
    os.makedirs(dir_name)


f = open(dir_name+"/info.txt", "w")
f.write("{\n")
for k in data.keys():
    f.write("'{}':'{}'\n".format(k, data[k]))
f.write("}")
f.close()

print(dir_name)

In [ ]:

optimizer = tfa.optimizers.AdamW(learning_rate=learning_rate, weight_decay=weight_decay)
vit_classifier.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy")],
)
history = vit_classifier.fit(
    x=x_train,
    y=y_train,
    batch_size=batch_size,
    epochs=num_epochs,
    validation_split=0.1,
)


In [ ]:

# Save the model
save_dir = '.../' + str(folder_num) + '/saved_model/'
os.makedirs(save_dir, exist_ok=True)
vit_classifier.save(os.path.join(save_dir, 'my_model.h5'))
# Load the saved model with custom objects

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


image_size = 224  # You can adjust this based on your model's input size



loaded_model = tf.keras.models.load_model(os.path.join('D:/Practice/Result/29__31_3_2024/BUSI_RoI/saved_model', 'my_model.h5'), custom_objects={"Patches": Patches, "PatchEncoder": PatchEncoder})

class_labels = ['Malignant', 'Benign']

# Function to make predictions on all images in a folder and calculate accuracy
def predict_and_calculate_accuracy(folder_path):
    try:
        if not os.path.isdir(folder_path):
            print(f"The provided path is not a valid directory: {folder_path}")
            return

     
        class_folders = [os.path.join(folder_path, class_name) for class_name in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, class_name))]
        num_classes = len(class_folders)

        if num_classes == 0:
            print(f"No subdirectories (classes) found in the folder: {folder_path}")
            return

        correct_predictions = 0
        total_predictions = 0

        true_labels = []
        predicted_labels = []

        for class_folder in class_folders:
            class_name = os.path.basename(class_folder)

            # Get a list of image files in the class folder
            image_files = [f for f in os.listdir(class_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp'))]

            for image_file in image_files:
                image_path = os.path.join(class_folder, image_file)

                # Read the image and preprocess it
                img = cv2.imread(image_path)
                if img is None:
                    print(f"Could not read the image: {image_path}")
                    continue

                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB format
                resized_img = cv2.resize(img, (image_size, image_size))
                normalized_img = resized_img / 255.0  # Normalize the image

                # Make the prediction
                prediction = loaded_model.predict(np.array([normalized_img]))

                # Get the predicted class index
                predicted_class_index = np.argmax(prediction)
                predicted_label = class_labels[predicted_class_index]

                true_labels.append(class_name)
                predicted_labels.append(predicted_label)

                # Check if the predicted class matches the image folder name
                if class_name.lower() == predicted_label.lower():
                    correct_predictions += 1

                total_predictions += 1

        # Calculate accuracy
        accuracy = (correct_predictions / total_predictions) * 100
        print(f"Total Accuracy: {accuracy:.2f}%")

        # Calculate confusion matrix and print classification report
        unique_labels = sorted(list(set(true_labels).union(set(predicted_labels))))
        cm = confusion_matrix(true_labels, predicted_labels, labels=unique_labels)
        report = classification_report(true_labels, predicted_labels, target_names=unique_labels)

        print("Confusion Matrix:")
        print(cm)
        print("Classification Report:")
        print(report)

    except Exception as e:
        print(f"An error occurred: {str(e)}")

# Provide the path to the folder containing the images you want to predict
folder_path = '...../test'
predict_and_calculate_accuracy(folder_path)
